# Working with HDF5
This notebook illustrates how to work with MkTurk PSTH data in HDF5 format. HDF5s allow you to slice data stored on disk, precluding the need to load all of the data from a given session before working with it. In applications where only a subset of the data are needed (e.g., only trials from a certain scenefile), this can reduce read times several times over, especially if the selected slices are contiguous on disk.

By way of preliminaries, let's load some necessary modules:

In [1]:
import os
import time
import numpy as np
from data_analysis_tools_mkTurk.utils_meta import get_recording_path
from data_analysis_tools_mkTurk.general import df_2_psth_mat
from data_analysis_tools_mkTurk.IO import ch_dicts_2_h5, h5_2_trial_df, h5_2_df

Also, use the cell below to set the prefix used to access Engram on the local system:

In [2]:
prefix = 'X:' # Edit this to whatever prefix is used to access Engram as a share/network drive on the local system 
mnt_path = prefix + os.path.sep 
base_data_path = os.path.join(mnt_path, 'Data')

      
<br/>

## 1. Writing data to HDF5
In this first part, we'll convert pickeld dicts of trial parameters and spike count data into HDF5 format, which we'll then read from in the subsequent section.   


<br/>

<u>**Prerequisites**</u><br/>
This write portion of the pipeline assumes that ephys data from SpikeGLX and behavioral data from MkTurk have already been preprocessed and registered to each other using the code illustrated in notebooks `get_data_dict_from_mkturk.ipynb`, `Demo.ipynb`, `Demo_2.ipynb`, and `Demo3.ipynb`. This should result in the following files for any given session:

- `data_dict_<session>`, where `<session>` stands for the name of the directory where the raw data for the session are saved.
- `stim_info_sess`, a pickled dict of general information about the stimuli shown in the session.
- `ch<iii>_psth_stim_meta`, a pickled dict including timing information about the peristimulus window for each stimulus, for each channels `<iii>`. 
- `ch<iii>_psth_stim`, a pickled dict of spike count data for each channels `<iii>`.

For an example of what preprocessed data for a given session should look like, see `axon.rc.zi.columbia.edu/mnt/smb/locker/issa-locker/users/Dan/ephys/West/West_20231109_R_H00_P46`. Let's use that session as an example here:

In [4]:
monkey = 'West'
date = '20231109'

# Define output directory:
recording_path = get_recording_path(base_data_path, monkey, date, depth=4)[0]preprocessed_data_path = os.path.join(mnt_path, 'users', 'Dan', 'ephys', monkey, recording_path.split(os.path.sep)[3])  
output_directory = os.path.join(mnt_path, 'users', 'Dan', 'ephys', monkey, recording_path.split(os.path.sep)[3], 'hdf_demo', 'write')

X:\Data\West


<br/>
<br/>

<u>**Choosing chunk size**</u><br/>
When working with HDF5s, data are loaded in "chunks," or sets of contiguous memory addresses. The chunk size determines how many such memory addresses are loaded at once. Perhaps counterintiuitively, despite the fact that this parameter relates to read behavior, it has to be set when the file is written. **Choosing an appropriate chunk size is really important!** The difference between a good and a bad chunk size can be a >100x difference in read/write times! In the past a chunk size of 100 has worked pretty well for MkTurk data. Also, since spike counts are always integers, we can save a little time and storage space by specifying that the datatype should be `int`:    

In [5]:
# Choose HDF5 params:
chunk_size = 100
dtype = int 

 <br/>
 <br/>
 
<u>**Writing HDF5 to disk**</u><br/>
We're almost ready to convert the preprocessed data to HDF5. First though, for demonstration/debugging purposes, we can also select a subset of channels to save data for, which will just make the demo much shorter. Let's just choose the first `max_channel` channels here:

In [6]:
# Choose channels to save data for:
max_channel = 4
channels = np.arange(max_channel)

In this case the channels happen to be consecutively numbered, but note in principle it's possible to select any arbitrary array (or list) of channel indices. Now we're ready to convert the preprocessed data to HDF5 using the `ch_dicts_2_h5()` function:

In [7]:
# Save preprocessed data as HDF5:
start = time.time() 
ch_dicts_2_h5(base_data_path, monkey, date, preprocessed_data_path, channels=channels, chunk_size=chunk_size, dtype=dtype, 
    save_output=True, output_directory=output_directory)
stop = time.time()
print('... done.')

X:\Data\West


X:\users\Dan\code\data_analysis_tools_mkTurk\IO.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  trial_params_df['idx_merge'] = np.arange(n_rows)


Generating dataframe of stimulus conditions...
X:\Data\West
X:\Data\West
Loading data for channel 1 of 4...
Loading data for channel 2 of 4...
Loading data for channel 3 of 4...
Loading data for channel 4 of 4...
Saving HDF5 to disk...
... done.


The amount of time this step takes depends on where you're running the code/where you're writing to. If writing from Axon to Engram, saving an entire session should take under a minute; if saving from a laptop to Engram over Columbia private WiFi, about 5-12 minutes; over the public Internet (via VPN), upwards of an hour. This sounds like a lot, but the idea is that you only have to do it once (or at least rarely) and then it'll pay off in the longer run with all the time you save loading in the future. 

In [8]:
print('Write duration = {} minutes'.format( (stop-start)/60 ))

Write duration = 2.572272205352783 minutes


<br/>

<u>**A caveat about time bins**</u></br>
Note that HDF5 stores data in an array-like structure, meaning that PSTHs for all stimuli must be of uniform shape -- meaning in turn that they must all have the same number of time bins. However, also note that different stimuli within a session can be of different duration (e.g., RSVP44 vs freeview stimuli). To accommodate these differences, `ch_dicts_2_h5()` pads PSTHs for all stimuli to be equal in duration to the max PSTH duration across the whole session. In other words, the number of time bins for every stimulus is made to equal the number of time bins for the longest stimulus. Given a session where the maximum stim duration is _v_ time bins, any shorter stimulus _u_ bins in duration will be aligned to the _start_ of the peristimulus window so that only the first _u_ out of _v_ bins will include spike data. Any excess time bins will be all nan. 


<br/>

# 2. Reading data from HDF5
Now that the preprocessed data have been saved to an HDF5, we can try reading it into RAM. 

<br/>

<u>**Pre-selecting trials**</u><br/>
Recall one of the useful things about HDF5 is that we can retrieve specific slices of data from disk without having to load the entire file. In practice, this often means loading the data for a subset of trials. Suppose we only want trials from scenefiles `neural_stim_5_1ABC_00` and `neural_stim_5_2UVW_03`:  

In [9]:
h5_path = os.path.join(mnt_path, 'users', 'Dan', 'ephys', monkey, recording_path.split(os.path.sep)[3], 'hdf_demo', 'read', 'all_psth.h5')
scenefiles = ['/mkturkfiles/scenebags/West/neural_stim_5_1ABC_00.json', 
              '/mkturkfiles/scenebags/West/neural_stim_5_2UVW_03.json']

Note here we're loading from a previously-written HDF5 containing data for all channels, in order to demonstrate read times for a more realistically-sized dataset. In order to retrive just the responses to these scenefiles, we first need to know which trials/RSVP stim are associated with each one. To check this, we can first load just the trial parameters (without the actual PSTH data) using the function `h5_2_trial_df()`:

In [10]:
# Load trial parameters without PSTH data:
trial_params = h5_2_trial_df(h5_path)

C:\Users\danie\anaconda3\envs\psth\Lib\site-packages\tables\attributeset.py:290: DataTypeWarning: Unsupported type for attribute 'scenefile_by_stim_mat' in node '/'. Offending HDF5 class: 8
  value = self._g_getattr(self._v_node, name)


This will return a pandas dataframe `trial_params` with columns `trial_num`, `rsvp_num`, `stim_id`, and `scenefile`, where each row corresponds to an individual RSVP stimulus presentation. (We can also get a more complete dataframe of all trial parameters by passing `params='all'` to `h5_2_trial_df()`, although the default subset of trial parameters is sufficient for many applications). We can then search for rows (stimulus presentations) associated with our scenefiles of interest using the standard constructions for querying dataframes:  

In [11]:
# Select stimulus presentations associated with requested scenefiles:
filter = trial_params.scenefile.isin(scenefiles)
rsvp44_trials = trial_params[filter]

<br/>
<br/>

<u>**Fetching spike data**</u><br/>
Now that we know which trials we want spike counts for, we can use the `h5_2_df()` function along with our dataframe of selected trials to load PSTHs just for those trials. Suppose moreover that we just want data from a certain time window and from certain channels:

In [12]:
# Read spike count data from HDF5 for requested trials:
time_window = [-0.1, 0.25] # Beginning and end of peristimulus time window for each stim, relative to trigger in seconds   
channels = np.arange(max_channel-1) # Select a subset of channels for demo purposes
rsvp44_data = h5_2_df(h5_path, trials=rsvp44_trials, time_window=time_window, channels=channels) # Fetch PSTHs

Pre-fetching PSTHs from HDF5...
... done.
Duration=3.738640081882477 minutes
Fancy slicing numpy array...
... done.
Duration=0.007328236103057861 minutes


Note this new dataframe `rsvp44_data` includes an additional column `psth` containing actual binned spike count data. For each row, the value of `psth` is a _c_-by-_t_ numpy array of binned spike counts, where _c_ is the number of channels in the loaded data, and _t_ is the number of time bins in the requested peri-stimulus time window. <br/>

Alternatively, `trials` can be an _s_-by-2 numpy array of trial/RSVP indices, where the first column consists of trial numbers and the second column consists of RSVP numbers within the corresponding trial. If `trials`, `time_window`, or `channels` are left unset, then all data for all trials, time bins, and channels will be returned, respectively.<br/>

To extract just the spike data into a numpy array, use

In [13]:
# Extract spike count data as array:
spikes = df_2_psth_mat(rsvp44_data)

where `spikes` will be _c_-by-_t_-by-_s_, where _s_ is the number of individual stimulus presentations in the input dataframe (i.e number of rows). 

Last updated DDK 2024-03-13.